In [138]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import sklearn
import torch
import torch.nn as nn
%matplotlib inline

In [139]:
image_path = '../9520-final-project/SUFRData/image_files/uniform_bg/scaling'

# Torchvision models

In [3]:
import torchvision.models as models
from torchvision.models import VGG

In [4]:
vgg19 = models.vgg19(pretrained=True)

In [5]:
resnet101 = models.resnet101(pretrained=True)

In [155]:
from cornet_S import CORnet_S
cornet = CORnet_S()

### Remove last layer from network architecture

In [6]:
vgg19.classifier = nn.Sequential(*list(vgg19.classifier.children())[:-1])

In [7]:
resnet101 = nn.Sequential(*list(resnet101.children())[:-1])

# Preprocess Images

In [8]:
from torchvision import transforms
from PIL import Image

def load_img(image_path, img_idx):
    return Image.open('{}/{}.jpg'.format(image_path, img_idx))

def process_img(img):    
    transform = transforms.Compose([
     transforms.Resize(256),                    
     transforms.CenterCrop(224),
     transforms.ToTensor(),
     transforms.Normalize(
         mean=[0.485, 0.456, 0.406],
         std=[0.229, 0.224, 0.225]                  
    )])
    img = transform(img)
    processed_img = torch.unsqueeze(img, 0)
    
    return processed_img

# Process Dataset

## TODO: make datasets with specific scale differences to see if there is difference with more scale variance

In [9]:
def parse_dataset(relative_path):
    f = open(relative_path, "r")
    dataset_idxs = []
    for line in f.readlines():
        img1_idx, img2_idx, label = line.strip().split(" ")
        dataset_idxs.append(((img1_idx, img2_idx), label))
    return dataset_idxs

def get_negative_pairs(face_idxs, c_offset_values):
    negative_pairs = []
    for c in c_offset_values:
        for i in face_idxs:
            for j in face_idxs:
                # ensures i != j and that there are no duplicate pairs
                if i > j: 
                    pair_idxs = (1+5*i+2,1+5*j+2+c)
                    negative_pairs.append(pair_idxs)
    return negative_pairs

def get_dataset_idxs(face_idxs, c_offset_values, num_negative_pairs=None):
    if num_negative_pairs is None:
        num_negative_pairs = face_idxs.shape[0] * len(c_offset_values)
    positive_pairs = np.array([(1+5*i+2,1+5*i+2+c) for c in c_offset_values for i in face_idxs])
    all_negative_pairs = np.array(get_negative_pairs(face_idxs, c_offset_values))
    negative_pairs = all_negative_pairs[list(np.random.choice(len(all_negative_pairs), num_negative_pairs, replace=False)), :]
    return positive_pairs, negative_pairs

#### Train and Test Dataset

In [10]:
train_face_idxs = np.random.choice(400, 350, replace=False)

mask = np.ones(400, dtype=bool)
mask[train_face_idxs] = False
test_face_idxs = np.arange(400)[mask]
c_offset_values = [-2,-1,1,2]

train_positive_pairs, train_negative_pairs = get_dataset_idxs(train_face_idxs, c_offset_values)
test_positive_pairs, test_negative_pairs = get_dataset_idxs(test_face_idxs, [-1,1])

#### Cosine Similarity for Evaluation

In [11]:
def cos_sim(a,b):
    dot = np.dot(a, b.T)
    norm_product = np.linalg.norm(a)*np.linalg.norm(b)
    return dot / norm_product

def compare_img_pairs(model, train_idxs, use_corr=False):
    similarities = []
    i = 0
    
    for img1_idx, img2_idx in train_idxs:
        img1 = process_img(load_img(image_path, img1_idx))
        img2 = process_img(load_img(image_path, img1_idx))
        vector1 = model(img1).detach().numpy()
        vector2 = model(img2).detach().numpy()
        
        if use_corr:
            similarities.append(np.corrcoef(vector1, vector2)[0][1])
        else:
            similarities.append(cos_sim(vector1, vector2)[0][0])
#         print(i)
        i += 1
        
    return similarities

In [58]:
positive_pair_similarities = compare_img_pairs(vgg19, test_positive_pairs, use_corr=False)
negative_pair_similarities = compare_img_pairs(vgg19, test_negative_pairs, use_corr=False)

In [59]:
np.array(positive_pair_similarities).mean()

0.4096328

In [60]:
np.array(negative_pair_similarities).mean()

0.41143674

# Precompute and Store Img. Feature Vectors

In [134]:
import json

vgg19_img_idx_to_vec = {}

for img_idx in range(1, 2001):
    img_vec = process_img(load_img(image_path, img_idx))
    img_feature_vec = vgg19.forward(img_vec).detach().squeeze().numpy()
    vgg19_img_idx_to_vec[img_idx] = img_feature_vec.tolist()
    if img_idx % 100 == 0:
        print("Processed {} images.".format(img_idx))

with open('./vgg19-feature-vecs.json', 'w') as f:
    json.dump(vgg19_img_idx_to_vec, f)

Processed 100 images.
Processed 200 images.
Processed 300 images.
Processed 400 images.
Processed 500 images.
Processed 600 images.
Processed 700 images.
Processed 800 images.
Processed 900 images.
Processed 1000 images.
Processed 1100 images.
Processed 1200 images.
Processed 1300 images.
Processed 1400 images.
Processed 1500 images.
Processed 1600 images.
Processed 1700 images.
Processed 1800 images.
Processed 1900 images.
Processed 2000 images.


In [141]:
def store_img_feature_vecs(model, modelname):
    model_img_idx_to_vec = {}

    for img_idx in range(1, 2001):
        img_vec = process_img(load_img(image_path, img_idx))
        img_feature_vec = model.forward(img_vec).detach().squeeze().numpy()
        model_img_idx_to_vec[img_idx] = img_feature_vec.tolist()
        if img_idx % 100 == 0:
            print("Processed {} images.".format(img_idx))

    with open('./{}-feature-vecs.json'.format(modelname), 'w') as f:
        json.dump(model_img_idx_to_vec, f)

In [143]:
store_img_feature_vecs(resnet101, 'resnet101')

Processed 100 images.
Processed 200 images.
Processed 300 images.
Processed 400 images.
Processed 500 images.
Processed 600 images.
Processed 700 images.
Processed 800 images.
Processed 900 images.
Processed 1000 images.
Processed 1100 images.
Processed 1200 images.
Processed 1300 images.
Processed 1400 images.
Processed 1500 images.
Processed 1600 images.
Processed 1700 images.
Processed 1800 images.
Processed 1900 images.
Processed 2000 images.


In [160]:
store_img_feature_vecs(cornet, 'cornet')

Processed 100 images.
Processed 200 images.
Processed 300 images.
Processed 400 images.
Processed 500 images.
Processed 600 images.
Processed 700 images.
Processed 800 images.
Processed 900 images.
Processed 1000 images.
Processed 1100 images.
Processed 1200 images.
Processed 1300 images.
Processed 1400 images.
Processed 1500 images.
Processed 1600 images.
Processed 1700 images.
Processed 1800 images.
Processed 1900 images.
Processed 2000 images.


In [12]:
import json

with open('./vgg19-feature-vecs.json') as f:
    vgg_img_idx_to_vec = json.load(f)

def get_vgg_feature_vec(img_idx):
    return vgg_img_idx_to_vec[str(img_idx)]

In [65]:
import json

with open('./resnet101-feature-vecs.json') as f:
    resnet_img_idx_to_vec = json.load(f)

def get_resnet_feature_vec(img_idx):
    return resnet_img_idx_to_vec[str(img_idx)]

In [161]:
import json

with open('./cornet-feature-vecs.json') as f:
    cornet_img_idx_to_vec = json.load(f)

def get_cornet_feature_vec(img_idx):
    return cornet_img_idx_to_vec[str(img_idx)]

## Implementation 0: Similarity-based classifier

### AUC Score (baseline)

In [61]:
from sklearn.metrics import roc_auc_score

labels = np.concatenate((np.zeros(len(negative_pair_similarities)), np.ones(len(positive_pair_similarities))))
roc_auc_score(labels, negative_pair_similarities + positive_pair_similarities)

0.49179999999999996

## Implementation 1: MLP Same/Different Classifier 

In [63]:
def mlp_preprocessing(model, positive_pairs, negative_pairs):

    pairs = np.concatenate((positive_pairs, negative_pairs))

    x = []
    for img1_idx, img2_idx in pairs:
        img1_feature_vec = get_vgg_vec(img1_idx, vgg_img_idx_to_vec, model, image_path)
        img2_feature_vec = get_vgg_vec(img2_idx, vgg_img_idx_to_vec, model, image_path)
        x_i = torch.cat([img1_feature_vec, img2_feature_vec], dim=1).detach().squeeze().numpy()
        x.append(x_i)

    x = np.array(x)
    y = np.concatenate((np.zeros(len(negative_pairs)), np.ones(len(positive_pairs))))
    
    return x, y

In [47]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=[512,64,10])
train_x, train_y = mlp_preprocessing(vgg19, train_positive_pairs[:10], train_negative_pairs[:10])
print(train_x.shape)
print(train_y.shape)

(20, 8192)
(20,)


In [50]:
model = mlp.fit(train_x,train_y)
model.best_loss_

0.0020002258602114507

In [51]:
test_x, test_y = mlp_preprocessing(vgg19, test_positive_pairs, test_negative_pairs)
model.score(test_x, test_y)

0.5

### Implementation 2: MLP Multiclass Classification and Correlation of feature vector for evaluation

In [104]:
from itertools import chain

train_faces = list(chain.from_iterable(train_positive_pairs)) + list(chain.from_iterable(train_negative_pairs))
train_faces_indices = np.unique(train_faces)
train_faces_labels = [(i-1) // 5 for i in train_faces_indices]
print(len(train_faces_indices))
print(len(train_faces_labels))

1750
1750


In [13]:
train_faces_indices = np.arange(1,1001)
train_faces_labels = [(i-1) // 5 for i in train_faces_indices]

In [144]:
def mlp2_preprocessing(modelname, train_faces_indices):
    x = []
    for i in train_faces_indices:
        if modelname == 'vgg19':
            img_feature_vec = get_vgg_feature_vec(i)
        elif modelname == 'resnet101':
            img_feature_vec = get_resnet_feature_vec(i)
        elif modelname == 'cornet':
            img_feature_vec = get_cornet_feature_vec(i)
        
        x.append(img_feature_vec)

    x = np.array(x)
    
    return x

In [106]:
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

mlp2_vgg = MLPClassifier(hidden_layer_sizes=[512,64,10])

train_faces_vgg = mlp2_preprocessing('vgg19', train_faces_indices)

In [107]:
len(train_faces_vgg)

1750

In [120]:
len(train_faces_vgg[0])

4096

In [108]:
model2_vgg = mlp2_vgg.fit(train_faces_vgg, train_faces_labels)
model2_vgg.best_loss_

0.004188641499188362

In [109]:
# test_positive_pairs_model2 = [pair for pair in test_positive_pairs if pair[0] > 1000 and pair[1] > 1000]
# test_negative_pairs_model2 = [pair for pair in test_negative_pairs if pair[0] > 1000 and pair[1] > 1000]
test_positive_pairs_model2 = [pair for pair in test_positive_pairs]
test_negative_pairs_model2 = [pair for pair in test_negative_pairs]

In [110]:
from sklearn.neural_network._base import ACTIVATIONS

def deepest_layer(data, MLP, layer=0):
    L = ACTIVATIONS['relu'](np.matmul(data, MLP.coefs_[layer]) + MLP.intercepts_[layer])
    layer += 1
    if layer >= len(MLP.coefs_)-1:
        return L
    else:
        return deepest_layer(L, MLP, layer=layer)

In [111]:
L_pos = []
L_neg = []

for pair in test_positive_pairs_model2:
    L = deepest_layer(mlp2_preprocessing('vgg19', pair), model2_vgg)
    L_pos.append(cos_sim(L[0], L[1]))

for pair in test_negative_pairs_model2:
    L = deepest_layer(mlp2_preprocessing('vgg19', pair), model2_vgg)
    L_neg.append(cos_sim(L[0], L[1]))

In [112]:
print(np.array(L_pos).mean())
print(np.array(L_neg).mean())

0.8365904917806316
0.7852289951762258


In [113]:
len(L_pos)

100

In [114]:
len(L_neg)

100

In [115]:
from sklearn.metrics import roc_auc_score

labels = np.concatenate((np.zeros(len(L_neg)), np.ones(len(L_pos))))
roc_auc_score(labels, L_neg + L_pos)

0.6412

#### Resnet

In [116]:
from sklearn.neural_network import MLPClassifier

mlp2_resnet = MLPClassifier(hidden_layer_sizes=[512,64,10])

train_faces_resnet = mlp2_preprocessing('resnet101', train_faces_indices)

In [121]:
len(train_faces_resnet[0])

2048

In [117]:
model2_resnet = mlp2_resnet.fit(train_faces_resnet, train_faces_labels)
model2_resnet.best_loss_

5.860516572023362

In [124]:
model2_resnet.classes_

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  32,  33,  34,  35,  36,  37,  38,  39,  41,
        42,  43,  44,  47,  49,  50,  51,  52,  54,  55,  56,  57,  58,
        59,  60,  61,  62,  65,  66,  67,  68,  69,  70,  72,  73,  74,
        75,  76,  77,  78,  79,  80,  81,  82,  83,  84,  86,  87,  88,
        89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101,
       102, 103, 104, 105, 107, 108, 109, 110, 111, 112, 113, 114, 115,
       116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128,
       130, 131, 132, 133, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157,
       158, 159, 160, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172,
       173, 174, 175, 176, 178, 179, 180, 181, 182, 183, 184, 185, 187,
       188, 189, 190, 191, 192, 193, 194, 196, 197, 198, 199, 20

In [126]:
model2_resnet.score(train_faces_resnet, train_faces_labels)

0.002857142857142857

In [118]:
L_pos = []
L_neg = []

for pair in test_positive_pairs_model2:
    L = deepest_layer(mlp2_preprocessing('resnet101', pair), model2_resnet)
    L_pos.append(cos_sim(L[0], L[1]))

for pair in test_negative_pairs_model2:
    L = deepest_layer(mlp2_preprocessing('resnet101', pair), model2_resnet)
    L_neg.append(cos_sim(L[0], L[1]))

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in double_scalars
  after removing the cwd from sys.path.


In [94]:
print(np.array(L_pos).mean())
print(np.array(L_neg).mean())

nan
nan


In [95]:
from sklearn.metrics import roc_auc_score

labels = np.concatenate((np.zeros(len(L_neg)), np.ones(len(L_pos))))
roc_auc_score(labels, L_neg + L_pos)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

#### Cornet

In [163]:
from sklearn.neural_network import MLPClassifier

mlp2_cornet = MLPClassifier(hidden_layer_sizes=[512,64,10])

train_faces_cornet = mlp2_preprocessing('cornet', train_faces_indices)

In [167]:
model2_cornet = mlp2_cornet.fit(train_faces_cornet, train_faces_labels)
model2_cornet.best_loss_

5.860485835083151

In [169]:
model2_cornet.classes_

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  32,  33,  34,  35,  36,  37,  38,  39,  41,
        42,  43,  44,  47,  49,  50,  51,  52,  54,  55,  56,  57,  58,
        59,  60,  61,  62,  65,  66,  67,  68,  69,  70,  72,  73,  74,
        75,  76,  77,  78,  79,  80,  81,  82,  83,  84,  86,  87,  88,
        89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101,
       102, 103, 104, 105, 107, 108, 109, 110, 111, 112, 113, 114, 115,
       116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128,
       130, 131, 132, 133, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157,
       158, 159, 160, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172,
       173, 174, 175, 176, 178, 179, 180, 181, 182, 183, 184, 185, 187,
       188, 189, 190, 191, 192, 193, 194, 196, 197, 198, 199, 20

In [171]:
model2_cornet.score(train_faces_cornet, train_faces_labels)

0.002857142857142857

In [172]:
L_pos = []
L_neg = []

for pair in test_positive_pairs_model2:
    L = deepest_layer(mlp2_preprocessing('resnet101', pair), model2_resnet)
    L_pos.append(cos_sim(L[0], L[1]))

for pair in test_negative_pairs_model2:
    L = deepest_layer(mlp2_preprocessing('resnet101', pair), model2_resnet)
    L_neg.append(cos_sim(L[0], L[1]))

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in double_scalars
  after removing the cwd from sys.path.


In [173]:
print(np.array(L_pos).mean())
print(np.array(L_neg).mean())

nan
nan


In [174]:
from sklearn.metrics import roc_auc_score

labels = np.concatenate((np.zeros(len(L_neg)), np.ones(len(L_pos))))
roc_auc_score(labels, L_neg + L_pos)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').